In [29]:
import time

import numpy as np
from IPython.display import Image
from keras.callbacks import TensorBoard
from keras.layers import Dense
from keras.models import Sequential
from pyswarms.single.global_best import GlobalBestPSO
from pyswarms.utils.functions import single_obj as fx
from pyswarms.utils.plotters import plot_surface
from pyswarms.utils.plotters.formatters import Animator, Designer, Mesher
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import warnings
warnings.filterwarnings("ignore")

In [2]:
iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']
enc = OneHotEncoder()
Y = enc.fit_transform(y[:, np.newaxis]).toarray()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(
    X_scaled, Y, test_size=0.5, random_state=2)
n_features = X.shape[1]
n_classes = Y.shape[1]

In [3]:
def create_custom_model(input_dim, output_dim, nodes, n=1, name='model'):
    model = Sequential(name=name)
    for i in range(n):
        model.add(Dense(nodes, input_dim=input_dim, activation='relu'))
    model.add(Dense(output_dim, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model


In [46]:
n_layers = 1
model = create_custom_model(n_features, n_classes,
                            10, n_layers)
model.summary()

start_time = time.time()
print('Model name:', model.name)
history_callback = model.fit(X_train, Y_train,
                             batch_size=5,
                             epochs=400,
                             verbose=0,
                             validation_data=(X_test, Y_test)
                             )
score = model.evaluate(X_test, Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print("--- %s seconds ---" % (time.time() - start_time))

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 10)                50        
                                                                 
 dense_7 (Dense)             (None, 3)                 33        
                                                                 
Total params: 83
Trainable params: 83
Non-trainable params: 0
_________________________________________________________________
Model name: model
3/3 [==============================] - 0s 3ms/step - loss: 0.2111 - accuracy: 0.9333
Test loss: 0.2111494392156601
Test accuracy: 0.9333333373069763
--- 17.46715259552002 seconds ---


In [5]:
def get_shape(model):
    weights_layer = model.get_weights()
    shapes = []
    for weights in weights_layer:
        shapes.append(weights.shape)
    return shapes
def set_shape(weights,shapes):
    new_weights = []
    index=0
    for shape in shapes:
        if(len(shape)>1):
            n_nodes = np.prod(shape)+index
        else:
            n_nodes=shape[0]+index
        tmp = np.array(weights[index:n_nodes]).reshape(shape)
        new_weights.append(tmp)
        index=n_nodes
    return new_weights

In [43]:
start_time = time.time()
def evaluate_nn(W, shape,X_test=X_train, Y_test=Y_train):
    result = []
    for weights in W:
        model.set_weights(set_shape(weights,shape))
        score = model.evaluate(X_test, Y_test, verbose=0)
        result.append(1-score[1])
    return result

shape = get_shape(model)
x_max = 1.0 * np.ones(83)
x_min = -1.0 * x_max
bounds = (x_min, x_max)
options = {'c1': 0.4, 'c2': 0.8, 'w': 0.4}
optimizer = GlobalBestPSO(n_particles=25, dimensions=83,
                          options=options, bounds=bounds)
cost, pos = optimizer.optimize(evaluate_nn, 15, X_test=X_train, Y_test=Y_train,shape=shape)
model.set_weights(set_shape(pos,shape))
score = model.evaluate(X_test, Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print("--- %s seconds ---" % (time.time() - start_time))

2021-12-27 15:00:42,770 - pyswarms.single.global_best - INFO - Optimize for 15 iters with {'c1': 0.4, 'c2': 0.8, 'w': 0.4}
pyswarms.single.global_best: 100%|██████████|15/15, best_cost=0.0133
2021-12-27 15:01:02,524 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.013333320617675781, best pos: [ 0.27911979 -0.55757932  0.2766565   0.35975281  0.72009114  0.7119445
  0.5776927   0.52077264  0.67504513 -0.49985279  0.15561242  0.49470757
  0.07110754 -0.02335194 -0.13379514 -0.25769252  0.8009418  -0.09804629
 -0.348635    0.69365033  0.48507372 -0.46965597  0.28400862 -0.36275778
  0.40690129  0.39743728  0.20008803  0.44225043  0.14224802  0.37116319
 -0.72829762 -0.07267608  0.63930366  0.29554009  0.39164875 -0.47153567
  0.58262708  0.88472097  0.48328034  0.82359525  0.69474802  0.57502965
 -0.31953482 -0.16788879  0.3410064   0.67655062 -0.14132718  0.21596682
 -0.47949632  0.06507222  0.38206743  0.42513836  0.55036095  0.72995095
 -0.00268799 -0.225735

3/3 [==============================] - 0s 2ms/step - loss: 0.5631 - accuracy: 0.9467
Test loss: 0.56305330991745
Test accuracy: 0.9466666579246521
--- 19.83688712120056 seconds ---


In [40]:
m = Mesher(func=fx.sphere)
pos_history = [pos[:, :2] for pos in optimizer.pos_history]
pos3d = m.compute_history_3d(pos_history)
# Assuming we already had an optimizer ready
my_animator = Animator(repeat=False)
my_designer = Designer(figsize=(6, 6))
animation = plot_surface(pos3d, animator=my_animator, designer=my_designer)
# %%
animation.save('pso.gif', writer='pillowwritter', fps=6, )
Image(url='pso.gif')

2021-12-27 14:58:39,759 - matplotlib.animation - WARNING - MovieWriter pillowwritter unavailable; using Pillow instead.
2021-12-27 14:58:39,760 - matplotlib.animation - INFO - Animation.save using <class 'matplotlib.animation.PillowWriter'>
